Misspellings of curse words?  

When building vocab, consider punctuation. Tokenize it first? Right now it just splits on whitespace. Moses probably has something for this

In [3]:
from time import time


import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, vstack
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
np.random.seed(1234) # help reproducibility
data = 'data/jigsaw/train.csv'

In [4]:
olid = open(data)
df = pd.read_csv(olid, sep=',')
olid.close()
df = df.drop('id', 1)

#make a single column from the 6 types of offensive in the data
df['offensive'] = df[df.columns[1:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)
df = df.drop(df.columns[[1,2,3,4,5,6]],1)

#make the single offensive column 0 if all elements are 0 and 1 if any are 1.
df['offensive'] = df['offensive'].map(lambda x: 0 if all(y=='0' for y in x.split(',')) else 1)


In [5]:
data_top = df.head()
print(data_top)

                                        comment_text  offensive
0  Explanation\nWhy the edits made under my usern...          0
1  D'aww! He matches this background colour I'm s...          0
2  Hey man, I'm really not trying to edit war. It...          0
3  "\nMore\nI can't make any real suggestions on ...          0
4  You, sir, are my hero. Any chance you remember...          0


In [6]:
text = np.array(df['comment_text'])
offen = np.array(df['offensive'])

from sklearn.model_selection import train_test_split
text_train, text_test, offen_train, offen_test = train_test_split(text, offen, test_size=0.2, random_state=4)


In [13]:
#create word count and tf-idf features for the training and test sets
cv = CountVectorizer(stop_words='english',max_features=30000)
tv = TfidfVectorizer(min_df=0.0, max_df=0.80, sublinear_tf=True)

cv_text_features = cv.fit_transform(text_train)
tv_text_features = tv.fit_transform(text_train)

cv_test_features = cv.transform(text_test)
tv_test_features = tv.transform(text_test)

In [22]:
from sklearn.pipeline import FeatureUnion
combinedFeatures = FeatureUnion([
    ('bow' , CountVectorizer(stop_words='english',max_features=30000)),
    ('tf-idf', TfidfVectorizer(min_df=0.0, max_df=0.80, sublinear_tf=True))])

text_features = combinedFeatures.fit_transform(text_train)
test_features = combinedFeatures.transform(text_test)

In [25]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2', max_iter=100, C=1)

In [28]:
# Logistic Regression model on TF-IDF features
lr_tfidf_predictions = train_predict_model(classifier=lr, train_features=text_features, train_labels=offen_train,
                                           test_features=test_features, test_labels=offen_test)

display_model_performance_metrics(true_labels=offen_test, predicted_labels=lr_tfidf_predictions,
                                      classes=[0,1])

Model Performance metrics:
------------------------------
Accuracy: 0.9567
Precision: 0.9548
Recall: 0.9567
F1 Score: 0.9537

Model Classification report:
------------------------------
              precision    recall  f1-score   support

           0       0.96      0.99      0.98     28657
           1       0.89      0.66      0.76      3258

    accuracy                           0.96     31915
   macro avg       0.93      0.82      0.87     31915
weighted avg       0.95      0.96      0.95     31915


Prediction Confusion Matrix:
------------------------------
          Predicted:      
                   0     1
Actual: 0      28393   264
        1       1118  2140


/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:60: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead


In [29]:
# Logistic Regression model on TF-IDF features
lr_tfidf_predictions = train_predict_model(classifier=lr, train_features=cv_text_features, train_labels=offen_train,
                                           test_features=cv_test_features, test_labels=offen_test)

display_model_performance_metrics(true_labels=offen_test, predicted_labels=lr_tfidf_predictions,
                                      classes=[0,1])

Model Performance metrics:
------------------------------
Accuracy: 0.9528
Precision: 0.9506
Recall: 0.9528
F1 Score: 0.949

Model Classification report:
------------------------------
              precision    recall  f1-score   support

           0       0.96      0.99      0.97     28657
           1       0.89      0.62      0.73      3258

    accuracy                           0.95     31915
   macro avg       0.92      0.80      0.85     31915
weighted avg       0.95      0.95      0.95     31915


Prediction Confusion Matrix:
------------------------------
          Predicted:      
                   0     1
Actual: 0      28402   255
        1       1251  2007


/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:60: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead


In [23]:
####Evaluation metrics
## I copied this from the article
## https://www.kaggle.com/divsinha/sentiment-analysis-countvectorizer-tf-idf

from sklearn import metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.base import clone
from sklearn.preprocessing import label_binarize
from scipy import interp
from sklearn.metrics import roc_curve, auc 


def get_metrics(true_labels, predicted_labels):
    
    print('Accuracy:', np.round(
                        metrics.accuracy_score(true_labels, 
                                               predicted_labels),
                        4))
    print('Precision:', np.round(
                        metrics.precision_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('Recall:', np.round(
                        metrics.recall_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('F1 Score:', np.round(
                        metrics.f1_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
                        

def train_predict_model(classifier, 
                        train_features, train_labels, 
                        test_features, test_labels):
    # build model    
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features) 
    return predictions    


def display_confusion_matrix(true_labels, predicted_labels, classes=[1,0]):
    
    total_classes = len(classes)
    level_labels = [total_classes*[0], list(range(total_classes))]

    cm = metrics.confusion_matrix(y_true=true_labels, y_pred=predicted_labels, 
                                  labels=classes)
    cm_frame = pd.DataFrame(data=cm, 
                            columns=pd.MultiIndex(levels=[['Predicted:'], classes], 
                                                  labels=level_labels), 
                            index=pd.MultiIndex(levels=[['Actual:'], classes], 
                                                labels=level_labels)) 
    print(cm_frame) 
    
def display_classification_report(true_labels, predicted_labels, classes=[1,0]):

    report = metrics.classification_report(y_true=true_labels, 
                                           y_pred=predicted_labels, 
                                           labels=classes) 
    print(report)
    
    
    
def display_model_performance_metrics(true_labels, predicted_labels, classes=[1,0]):
    print('Model Performance metrics:')
    print('-'*30)
    get_metrics(true_labels=true_labels, predicted_labels=predicted_labels)
    print('\nModel Classification report:')
    print('-'*30)
    display_classification_report(true_labels=true_labels, predicted_labels=predicted_labels, 
                                  classes=classes)
    print('\nPrediction Confusion Matrix:')
    print('-'*30)
    display_confusion_matrix(true_labels=true_labels, predicted_labels=predicted_labels, 
                             classes=classes)


def plot_model_decision_surface(clf, train_features, train_labels,
                                plot_step=0.02, cmap=plt.cm.RdYlBu,
                                markers=None, alphas=None, colors=None):
    
    if train_features.shape[1] != 2:
        raise ValueError("X_train should have exactly 2 columnns!")
    
    x_min, x_max = train_features[:, 0].min() - plot_step, train_features[:, 0].max() + plot_step
    y_min, y_max = train_features[:, 1].min() - plot_step, train_features[:, 1].max() + plot_step
    xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))

    clf_est = clone(clf)
    clf_est.fit(train_features,train_labels)
    if hasattr(clf_est, 'predict_proba'):
        Z = clf_est.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:,1]
    else:
        Z = clf_est.predict(np.c_[xx.ravel(), yy.ravel()])    
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, Z, cmap=cmap)
    
    le = LabelEncoder()
    y_enc = le.fit_transform(train_labels)
    n_classes = len(le.classes_)
    plot_colors = ''.join(colors) if colors else [None] * n_classes
    label_names = le.classes_
    markers = markers if markers else [None] * n_classes
    alphas = alphas if alphas else [None] * n_classes
    for i, color in zip(range(n_classes), plot_colors):
        idx = np.where(y_enc == i)
        plt.scatter(train_features[idx, 0], train_features[idx, 1], c=color,
                    label=label_names[i], cmap=cmap, edgecolors='black', 
                    marker=markers[i], alpha=alphas[i])
    plt.legend()
    plt.show()


def plot_model_roc_curve(clf, features, true_labels, label_encoder=None, class_names=None):
    
    ## Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    if hasattr(clf, 'classes_'):
        class_labels = clf.classes_
    elif label_encoder:
        class_labels = label_encoder.classes_
    elif class_names:
        class_labels = class_names
    else:
        raise ValueError('Unable to derive prediction classes, please specify class_names!')
    n_classes = len(class_labels)
    y_test = label_binarize(true_labels, classes=class_labels)
    if n_classes == 2:
        if hasattr(clf, 'predict_proba'):
            prob = clf.predict_proba(features)
            y_score = prob[:, prob.shape[1]-1] 
        elif hasattr(clf, 'decision_function'):
            prob = clf.decision_function(features)
            y_score = prob[:, prob.shape[1]-1]
        else:
            raise AttributeError("Estimator doesn't have a probability or confidence scoring system!")
        
        fpr, tpr, _ = roc_curve(y_test, y_score)      
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label='ROC curve (area = {0:0.2f})'
                                 ''.format(roc_auc),
                 linewidth=2.5)
        
    elif n_classes > 2:
        if hasattr(clf, 'predict_proba'):
            y_score = clf.predict_proba(features)
        elif hasattr(clf, 'decision_function'):
            y_score = clf.decision_function(features)
        else:
            raise AttributeError("Estimator doesn't have a probability or confidence scoring system!")

        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        ## Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        ## Compute macro-average ROC curve and ROC area
        # First aggregate all false positive rates
        all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
        # Then interpolate all ROC curves at this points
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(n_classes):
            mean_tpr += interp(all_fpr, fpr[i], tpr[i])
        # Finally average it and compute AUC
        mean_tpr /= n_classes
        fpr["macro"] = all_fpr
        tpr["macro"] = mean_tpr
        roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

        ## Plot ROC curves
        plt.figure(figsize=(6, 4))
        plt.plot(fpr["micro"], tpr["micro"],
                 label='micro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["micro"]), linewidth=3)

        plt.plot(fpr["macro"], tpr["macro"],
                 label='macro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["macro"]), linewidth=3)

        for i, label in enumerate(class_labels):
            plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                           ''.format(label, roc_auc[i]), 
                     linewidth=2, linestyle=':')
    else:
        raise ValueError('Number of classes should be atleast 2 or more')
        
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()

NameError: name 'lr' is not defined

In [47]:
lr_tfidf_predictions = train_predict_model(classifier=lr, train_features=cv_text_features, train_labels=offen_train,
                                           test_features=cv_test_features, test_labels=offen_test)

display_model_performance_metrics(true_labels=offen_test, predicted_labels=lr_tfidf_predictions,
                                      classes=[0,1])

Model Performance metrics:
------------------------------
Accuracy: 0.9528
Precision: 0.9506
Recall: 0.9528
F1 Score: 0.949

Model Classification report:
------------------------------
              precision    recall  f1-score   support

           0       0.96      0.99      0.97     28657
           1       0.89      0.62      0.73      3258

    accuracy                           0.95     31915
   macro avg       0.92      0.80      0.85     31915
weighted avg       0.95      0.95      0.95     31915


Prediction Confusion Matrix:
------------------------------
          Predicted:      
                   0     1
Actual: 0      28402   255
        1       1251  2007


/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:57: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:59: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
